<a href="https://colab.research.google.com/github/nnilayy/Unet/blob/main/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt 
import tensorflow as tf 
import numpy as np
import cv2 
import os 

In [2]:
!unzip -q /content/dataset.zip -d /content/

In [13]:
train=ImageDataGenerator(rescale=1/255)
validation=ImageDataGenerator(rescale=1/255)

In [21]:
train_dataset=train.flow_from_directory('/content/dataset/train',
                                        target_size=(224,224),
                                        batch_size=3,
                                        class_mode='binary')

val_dataset=validation.flow_from_directory('/content/dataset/validation',
                                        target_size=(224,224),
                                        batch_size=3,
                                        class_mode='binary')

Found 60 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [22]:
train_dataset.class_indices

{'basketball': 0, 'football': 1}

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, AveragePooling2D, Flatten, Dropout, BatchNormalization, InputLayer, Reshape
model = Sequential()
model.add(InputLayer(input_shape=(224, 224,3)))
# model.add(Reshape(target_shape=(224, 224, 1)))

model.add(Conv2D(16, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(16, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(Conv2D(16, 3, padding='same', activation="relu"))
model.add(MaxPooling2D(2, padding='same'))
model.add(MaxPooling2D(2, padding='same'))
model.add(MaxPooling2D(2, padding='same'))

model.add(Flatten())
model.add(Dense(32, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history=model.fit(train_dataset, steps_per_epoch=5, epochs=50, validation_data=val_dataset)

In [56]:
file="/content/images.jpg"
image=cv2.imread(file)
image=cv2.resize(image,(224,224))
image=np.expand_dims(image,0)
prediction=int(model.predict(image))

if prediction == 0:
  print('Its a basketball')
elif prediction==1:
  print('Its a football')

1/1 [==============================] - 0s 22ms/step
Its a football


In [ ]:
# https://www.youtube.com/watch?v=uqomO_BZ44g